### Цель исследования

- Компания купила крупный сервис для чтения книг по подписке.
- Задача — проанализировать базу данных.
- В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг.
- Эти данные помогут сформулировать ценностное предложение для нового продукта.

### Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [1]:
# импорты
import os
import pandas as pd
from sqlalchemy import text, create_engine
from dotenv import load_dotenv

load_dotenv()

# устанавливаем параметры
db_config = {'user': os.getenv('user', ''), # имя пользователя
'pwd': os.getenv('pwd', ''), # пароль
'host': os.getenv('host', ''),
'port': 6432, # порт подключения
'db': os.getenv('db_final', '')} # название базы данных

connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

def get_connection_db(query: str,
                      con = engine.connect())-> None:
    """Подключение к бд и получение инфы."""
    display(pd.io.sql.read_sql(sql=text(query), con = con))

In [2]:
# Исследуйте таблицы — выведите первые строки;

for table in ['books', 'authors', 'publishers', 'ratings', 'reviews']:
    print('Таблица: ', table)
    query = f'''SELECT * FROM {table} LIMIT 5;'''
    get_connection_db(query)

Таблица:  books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Таблица:  authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Таблица:  publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Таблица:  ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Таблица:  reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


In [3]:
# Посчитайте, сколько книг вышло после 1 января 2000 года;
query = """SELECT COUNT(book_id)
           FROM books
           WHERE publication_date::date > '01.01.2000';"""
get_connection_db(query)

,count
0,819


In [4]:
# Для каждой книги посчитайте количество обзоров и среднюю оценку;
query = """SELECT book_id,
                  title,
                  COUNT(DISTINCT review_id) AS num_reviews,
                  AVG(rating)::numeric(22,2) AS avg_rating 
           FROM books
           LEFT JOIN reviews USING(book_id)
           LEFT JOIN ratings USING(book_id)
           GROUP BY book_id
           ORDER BY num_reviews DESC,
                    avg_rating DESC;"""
get_connection_db(query)

,book_id,title,num_reviews,avg_rating
0,948,Twilight (Twilight #1),7,3.66
1,302,Harry Potter and the Prisoner of Azkaban (Harr...,6,4.41
2,299,Harry Potter and the Chamber of Secrets (Harry...,6,4.29
3,656,The Book Thief,6,4.26
4,734,The Glass Castle,6,4.21
...,...,...,...,...
995,191,Disney's Beauty and the Beast (A Little Golden...,0,4.00
996,387,Leonardo's Notebooks,0,4.00
997,221,Essential Tales and Poems,0,4.00
998,83,Anne Rice's The Vampire Lestat: A Graphic Novel,0,3.67


In [5]:
# Определите издательство, которое выпустило наибольшее число книг толще 50 страниц — так вы исключите из анализа брошюры;
query = """SELECT publisher,
                  COUNT(book_id) AS pub_books
           FROM publishers
           LEFT JOIN books USING(publisher_id)
           WHERE num_pages > '50'
           GROUP BY publisher
           ORDER BY pub_books DESC
           LIMIT '1';"""
get_connection_db(query)

,publisher,pub_books
0,Penguin Books,42


In [6]:
# Определите автора с самой высокой средней оценкой книг — учитывайте только книги с 50 и более оценками;
query = """WITH tt AS (SELECT book_id,
                              AVG(rating) AS avg_rating_book
                       FROM ratings
                       WHERE book_id IN (SELECT book_id
                                         FROM ratings
                                         GROUP BY book_id
                                         HAVING COUNT(ratings) >= 50)
                       GROUP BY book_id)
            SELECT AVG(avg_rating_book) AS avg_rating_author,
                   author
            FROM tt
            LEFT JOIN books USING(book_id)
            LEFT JOIN authors USING(author_id)
            GROUP BY author
            ORDER BY avg_rating_author DESC
            LIMIT 1;"""
get_connection_db(query)

,avg_rating_author,author
0,4.283844,J.K. Rowling/Mary GrandPré


In [7]:
# Посчитайте среднее количество обзоров от пользователей, которые поставили больше 48 оценок.
query = """SELECT AVG(tt1.num_rev)::int AS avg_rev
           FROM (SELECT COUNT(review_id) AS num_rev
                 FROM reviews
                 WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(rating_id) > 48)
                GROUP BY username) AS tt1;"""
get_connection_db(query)

,avg_rev
0,24


Выводы:
- `819` книг вышло после 1 января 2000 года
- разброс по рейтингам и по кол-ву отзывов достаточно большой
- издательство `Penguin Books`, выпустило наибольшее число книг `42` толще 50 страниц
- `J.K. Rowling/Mary GrandPré` автор с самой высокой средней оценкой книг
- `24` - среднее количество обзоров от пользователей, которые поставили больше 48 оценок